# Plot Halo Mergers
Nithun Selva, July 2025

### Import

In [ ]:
%matplotlib inline
import os
os.environ['TANGOS_SIMULATION_FOLDER'] = '/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/'
# os.environ['TANGOS_DB_CONNECTION'] = '/home/selvani/MAP/Data/Marvel_BN_N10.db'
os.environ['TANGOS_DB_CONNECTION'] = '/home/selvani/MAP/pynbody/Tangos/Marvel_BN_N10.db'
os.chdir('/home/selvani/MAP/pynbody/AnnaWright_startrace/')

import pynbody
import numpy as np
import h5py
import math
import tangos as db
import matplotlib.pyplot as plt
import glob
from pynbody.array import SimArray
import IonUtils
import pandas as pd

In [2]:
simpath = '/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/'
sims = 'cptmarvel.cosmo25cmb.4096g5HbwK1BH'

ss_dir = 'cptmarvel.4096g5HbwK1BH_bn'
sim_base = simpath + ss_dir + '/'
ss_z0 = sim_base + sims + '.004096'

odir = "/home/selvani/MAP/pynbody/stellarhalo_trace_aw/"

In [5]:
timestep = db.get_timestep(ss_dir+'/%'+str(4096))
timestep

<TimeStep 'cptmarvel.4096g5HbwK1BH_bn/cptmarvel.cosmo25cmb.4096g5HbwK1BH.004096' z=-0.00 t=13.73 Gyr>

In [10]:
halo = timestep.halos.first()

In [12]:
snap = halo.load()

In [23]:
snap.physical_units()

In [25]:
snap.base

<TipsySnap "/home/selvani/MAP/Sims/cptmarvel.cosmo25cmb/cptmarvel.cosmo25cmb.4096g5HbwK1BH/cptmarvel.4096g5HbwK1BH_bn/cptmarvel.cosmo25cmb.4096g5HbwK1BH.004096" len=89818099>

In [18]:
all_timesteps = db.get_simulation(ss_dir).timesteps

In [ ]:
# code to kinda replicate halo trace in tangos??
import pandas as pd

def trace_halos(halos, timesteps, save_file='halo_trace.h5'):
    uniqhaloid = [halo.halo_number for halo in halos]
    columns = [step.extension[-6:] for step in timesteps]
    columns.reverse()
    print(columns)
    
    df = pd.DataFrame(index=uniqhaloid, columns=columns)
    
    for halo in halos:
        print(f"Halo ID: {halo.halo_number}, Mass: {halo['Mvir']}, Stars: {halo.NStar}")
        halo_numbers, dbids = halo.calculate_for_progenitors("halo_number()", "dbid()")
        snapshots = [db.get_halo(dbid).timestep.extension[-6:] for dbid in dbids]
        print(f"\tProgenitor Halo Numbers: {halo_numbers}, Snapshot: {snapshots}\n")

        df.loc[halo.halo_number, snapshots] = halo_numbers

    # Replace NaN with -1 and change all types to int
    df = df.fillna(-1).astype(int)
    
    # Set the index to be the values from the '004096' column
    df.index = df['004096']
    df.index.name = '004096'
    df = df.drop(columns=['004096'])  # Drop the '004096' column as it's now the index
    
    if save_file:
        print('Saving to: ' + os.path.abspath(save_file))
        if os.path.isfile(save_file):
            print('File exists: ' + save_file)
            print('Aborting')
            return df
        df.to_hdf(save_file, key='ids')
    
    return df

In [ ]:
timestep = db.get_timestep("cptmarvel.4096g5HbwK1BH_bn/%4096")
all_halos = timestep.halos.all()
print("There are %d halos in the snapshot." % len(all_halos))

# Filter for halos with stars
halos_with_stars = [h for h in all_halos if h.NStar > 0]

In [ ]:
# Get all halos in 004096 snapshot
all_timesteps = db.get_simulation("cptmarvel.4096g5HbwK1BH_bn").timesteps
halos_stars_dict = {}
for timestep in all_timesteps:
    all_halos = timestep.halos.all()
    # print("There are %d halos in the snapshot." % len(all_halos))

    # Filter for halos with stars
    halos_with_stars = [h for h in all_halos if h.NStar > 0]
    print("Timestep: %s" % timestep.extension[-6:])
    print("There are %d halos with stars.\n" % len(halos_with_stars))
    halos_stars_dict[timestep.extension[-6:]] = halos_with_stars
    # for halo in halos_with_stars:
    #     print("Halo ID: %s, Mass: %1.2eM☉, Stars: %d" % (halo.halo_number, halo['Mvir'], halo.NStar))

In [ ]:
#show the movement of each halo's COM
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig= plt.figure(figsize=(12,9))
ax=fig.add_subplot(111, projection='3d')

for halo_id in df["halo_id"].unique():
    sub = df[df["halo_id"] == halo_id]
    ax.plot(sub["x_com"], sub["y_com"], sub["z_com"], label=f"Halo {halo_id}")

ax.set_xlabel("x(kpc)")
ax.set_ylabel("y(kpc)")
ax.set_zlabel("z(kpc)")

ax.set_title("Movement of COM")
ax.legend()
#plt.grid(True)
# plt.savefig('Movement_COM.png')
plt.show()